In [1]:
import os
import random
from os.path import join
from time import time

import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical
from torch import optim
from torch.nn import BCELoss
from torch.optim import Adam
from torch.utils.data import TensorDataset
from torchvision.utils import save_image

from models.de_OPPO import DAE
from models.dis_z import DIS_Z
from models.activity_recognition import *
from utils.function import *

# from matplotlib import pyplot as plt
# %pylab inline

In [2]:
class Args:
    def __init__(self):
        self.root = '../data'
        self.batchSize = 64
        self.maxEpochs = 100
        self.nz = 200
        self.lr = 1e-4
        self.fSize = 64
        self.outDir = 'data/experiments/DAE_EVAL_Missing'
        self.commit = 'eval'
        self.alpha = 1.0
        # self.sigma = 0.35
        self.M = 5
        self.loss = 'MSE' #'BCE'
        self.loadDAE = False
        self.loadSVM = False    
        self.load_DAE_from = None
        self.evalMode = False
        self.comment = ''
        self.momentum = 0.1
        self.c = 0.01
        self.svmLR = 1e-4
        self.Ntest = 100
        self.gpuNo = 2
        self.cuda_id = 2
        self.multimodalZ = False
        self.window_len = 512
        self.stride_len = 20
        self.act_list = [1, 2, 3, 4, 5, 6, 7, 12, 13, 16, 17, 24]
        self.imSize = 64
        # self.sigma = [50, 80]
        # self.sigma = 0.2
        # self.sigma= [0.1, 4, 10]
        self.sigma = [8, 10]
        # self.sigma = 0.2
        self.random_seed = 2
        self.train_split = 0.8

In [3]:
def build_dis(args, dae, multimodalZ):
    if not multimodalZ:
        print('\n ** USING NORMAL PRIOR **')
        prior = dae.norm_prior
        NZ = args.nz
    else:
        print('\n ** USING MULTIMODAL PRIOR **')
        prior = dae.multi_prior
        NZ = 2
    dis = DIS_Z(nz=NZ, prior=prior)

    return dis, NZ

In [4]:
# def prepare_data(args):
#     X=[]
#     user_labels=[]
#     act_labels=[]

#     # columns for IMU data
#     imu_locs = [4,5,6, 10,11,12, 13,14,15, 
#                 21,22,23, 27,28,29, 30,31,32, 
#                 38,39,40, 44,45,46, 47,48,49
#             ] 

#     scaler = MinMaxScaler()
#     # scaler = StandardScaler()

#     for uid in np.arange(1,10):
#         path = '../../../../../PAMAP2_Dataset/Protocol/subject10' + str(uid) + '.dat'
#         df = pd.read_table(path, sep=' ', header=None)
#         act_imu_filter = df.iloc[:, imu_locs] 

#         for act_id in range(len(args.act_list)):
#             act_filter =  act_imu_filter[df.iloc[:, 1] == args.act_list[act_id]]
#             act_data = act_filter.to_numpy()
                
#             act_data = np.transpose(act_data)
#             # sliding window segmentation
#             start_idx = 0
#             while start_idx + args.window_len < act_data.shape[1]:
#                 window_data = act_data[:, start_idx:start_idx+args.window_len]
#                 downsamp_data = window_data[:, ::3] # downsample from 100hz to 33.3hz
#                 downsamp_data = np.nan_to_num(downsamp_data) # remove nan

#                 X.append(downsamp_data)
#                 user_labels.append(uid)
#                 act_labels.append(act_id)
#                 start_idx = start_idx + args.stride_len

#     X_n = np.array(X).astype('float32')

#     normalized_X = np.zeros_like(X_n) # allocate numpy array for normalized data
#     for ch_id in range(X_n.shape[1]): # loop the 27 sensor channels
#         ch_data = X_n[:, ch_id, :] # the data of channel id
#         scaler = MinMaxScaler() # maybe different scalers?
#         ch_data = scaler.fit_transform(ch_data) # scale the data in this channel to [0,1]
#         normalized_X[:, ch_id, :] = ch_data # assign normalized data to normalized_X
#     #normalized_X = np.transpose(normalized_X, (0, 2, 1)) # I overwrote X here, changed dimensions into: num_samples, sequence_length, feature_length

#     normalized_X= normalized_X.reshape(normalized_X.shape[0], 1, normalized_X.shape[1], normalized_X.shape[2]) # convert list to numpy array
#     act_labels = np.array(act_labels).astype('float32')
#     act_labels = act_labels.reshape(act_labels.shape[0],1)
#     act_labels = to_categorical(act_labels, num_classes=len(args.act_list))

#     return normalized_X, act_labels

In [5]:
from utils.sliding_window import sliding_window
import pickle as cp

In [6]:
# Number of Sensor Channels used in the OPPORTUNITY dataset.
NB_SENSOR_CHANNELS = 113

# Number of classes in which data is classified (or to be classified).
NUM_CLASSES = 5

# Length of the sliding window used to segmenting the time-series-data.
SLIDING_WINDOW_LENGTH = 24

# Steps of the sliding window used in segmenting the data.
SLIDING_WINDOW_STEP = 12

act_labels_txt = ['std', 'wlk', 'sit', 'lie', 'null']

# Variable for Batch Size.
# BATCH_SIZE = 100

# Number filters used in convolutional layers.
# NUM_FILTERS = 64

# Size of filters used in convolutional layers.
# FILTER_SIZE = 5

# Units in the long short-term recurrent layers.
# NUM_UNITS_LSTM = 128

In [7]:
def load_dataset(filename):

    f = open(filename, 'rb')
    data = cp.load(f)
    f.close()

    X_train, y_train = data[0]
    X_test, y_test = data[1]

    print(" ..from file {}".format(filename))
    print(" ..reading instances: train {0}, test {1}".format(X_train.shape, X_test.shape))

    X_train = X_train.astype(np.float32)
    X_test = X_test.astype(np.float32)

    # The targets are casted to int8 for GPU compatibility.
    y_train = y_train.astype(np.uint8)
    y_test = y_test.astype(np.uint8)

    return X_train, y_train, X_test, y_test

print("Loading Data...")
X_train, y_train, X_test, y_test = load_dataset('../../../../../data/oppChallenge_gestures.data')

assert NB_SENSOR_CHANNELS == X_train.shape[1]
def opp_sliding_window(data_x, data_y, ws, ss):
    data_x = sliding_window(data_x,(ws,data_x.shape[1]),(ss,1))
    data_y = np.asarray([[i[-1]] for i in sliding_window(data_y,ws,ss)])
    return data_x.astype(np.float32), data_y.reshape(len(data_y)).astype(np.uint8)

# Sensor data is segmented using a sliding window mechanism
X_test, y_test = opp_sliding_window(X_test, y_test, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
print(" ..after sliding window (testing): inputs {0}, targets {1}".format(X_test.shape, y_test.shape))

# Data is reshaped since the input of the network is a 4 dimension tensor
X_test = X_test.reshape((-1, SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS))
X_test = np.transpose(X_test, (0, 2, 1))
X_test= X_test.reshape(X_test.shape[0], 1, X_test.shape[1], X_test.shape[2]) # convert list to numpy array

Loading Data...
 ..from file ../../../../../data/oppChallenge_gestures.data
 ..reading instances: train (557963, 113), test (118750, 113)
 ..after sliding window (testing): inputs (9894, 24, 113), targets (9894,)


In [8]:
X_train, y_train = opp_sliding_window(X_train, y_train, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
print(" ..after sliding window (training): inputs {0}, targets {1}".format(X_train.shape, y_train.shape))
X_train = X_train.reshape((-1,SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS))
X_train = np.transpose(X_train, (0, 2, 1))
X_train= X_train.reshape(X_train.shape[0], 1, X_train.shape[1], X_train.shape[2]) # convert list to numpy array

X_train.shape


 ..after sliding window (training): inputs (46495, 24, 113), targets (46495,)


(46495, 1, 113, 24)

In [9]:
(94895, 1, 27, 171)
(94895, 12)

(94895, 12)

In [10]:
y_train = to_categorical(y_train, num_classes=NUM_CLASSES)
y_test = to_categorical(y_test, num_classes=NUM_CLASSES)

In [11]:
def prepare_dataloaders(args, X_train, X_test, y_train, y_test):
    trainDataset = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
    testDataset = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))
    # Train/Test dataset split
    # train_size = int(args.train_split * len(dataset))
    # test_size = len(dataset) - train_size
    # # trainDataset, testDataset = torch.Tensor(dataset[:train_size]), torch.Tensor(dataset[train_size:])
    # trainDataset, testDataset = torch.utils.data.random_split(dataset, [train_size, test_size])

    # trainDataset2, testDataset2 = torch.utils.data.random_split(dataset, [train_size, test_size])

    trainLoader = torch.utils.data.DataLoader(trainDataset,
        batch_size=args.batchSize, shuffle=True) 

    testLoader = torch.utils.data.DataLoader(testDataset,
        batch_size=args.batchSize, shuffle=False)
    return trainLoader, testLoader

In [12]:
def train_dae(args, exDir, trainLoader, testLoader):
    dae = DAE(nz=args.nz, imSize=args.imSize, fSize=args.fSize, sigma=args.sigma, multimodalZ=args.multimodalZ) #sigma=level of corruption
    # dis, NZ = build_dis(args, dae=dae, multimodalZ=args.multimodalZ)
    
    if dae.useCUDA:
        torch.cuda.set_device(args.gpuNo)
        # torch.cuda.set_device(args.gpuNo)
        dae.cuda()
        # dis.cuda()
    
    save_input_args(exDir, args)  #save training opts
    save_exp_details(args, exDir)

    # #Create optimizers
    optimDAE = optim.RMSprop(dae.parameters(), lr = args.lr, momentum=args.momentum)
    # optimDIS = optim.RMSprop(dis.parameters(), lr = args.lr, momentum=args.momentum)

    # optimDAE = optim.Adam(dae.parameters(), lr = args.lr)
    # optimDIS = optim.Adam(dis.parameters(), lr = args.lr)    
    
    # #Keeping track of training
    losses = {'enc': [], 'rec': [], 'dis':[], 'test rec':[]}

    # xTest, yTest = prep_data(iter(testLoader).next(), useCUDA=dae.useCUDA)

    with torch.autograd.set_detect_anomaly(True):
        for e in range(args.maxEpochs):

            epochEncLoss=0
            epochRecLoss=0
            epochDisLoss=0
            epochKlLoss=0

            for i, data in enumerate(trainLoader):
                # print(i)

                T = time()

                dae.train()
                # dis.train()

                x, y = prep_data(data, useCUDA=dae.useCUDA)
                # print(x.shape)
            
                # get outputs
                zFake, xRec = dae.forward(x)

                # clac losses
                recLoss = dae.rec_loss(xRec, x, loss=args.loss)  #loss='BCE' or 'MSE'
                # klLoss = dae.kl_loss(z_m, z_l)
                # loss = recLoss + 0.1*klLoss
                
            
                optimDAE.zero_grad()
                # loss.backward()
                recLoss.backward()
                optimDAE.step()            
            
                # zFake, xRec = dae.forward(x)    
            
                # disLoss = dis.dis_loss(zFake)
                #do updates

                # optimDIS.zero_grad()
                # disLoss.backward()
                # optimDIS.step()
            
                # encLoss = dis.gen_loss(zFake)
            
                # optimDAE.zero_grad()
                # encLoss.backward()
                # optimDAE.step()                   


                # epochEncLoss+=encLoss.detach().cpu().numpy()
                epochRecLoss+=recLoss.detach().cpu().numpy()
                # epochKlLoss+=klLoss.detach().cpu().numpy()
                # epochDisLoss+=disLoss.detach().cpu().numpy()
            
                if i%100 == 0:
                    print('[%d, %d] rec: %0.5f, time: %0.3f' % (e, i, recLoss.detach().cpu().numpy(), time() - T))
                    # print('[%d, %d] rec: %0.5f, kl: %0.5f, time: %0.3f' % (e, i, recLoss.detach().cpu().numpy(), klLoss.detach().cpu().numpy(), time() - T))
                    

            # storing losses for plotting later
            # losses['enc'].append(epochEncLoss/i)
            losses['rec'].append(epochRecLoss/i)
            # losses['dis'].append(epochDisLoss/i)

            #### Test
            dae.eval()
            # dis.eval()

            # #get test outuputs and losses
            # xTest, yTest = prep_data(iter(testLoader).next(), useCUDA=dae.useCUDA)
            # zTest, recTest = dae.forward(xTest)  #N.B. corruption in here
            # recLossTest = dae.rec_loss(recTest, xTest)
            # # klLossTest = dae.kl_loss(z_m_test, z_l_test)
            # #Plot losses
            # # losses['test rec'].append(recLossTest.data[0])
            # losses['test rec'].append(recLossTest.detach().cpu().numpy())

            # if e > 0: #only one point for test rec otherwise
            #     plot_losses(losses, exDir, epochs=e+1)
            #     plot_norm_losses(losses, exDir)

            #save parameters
            dae.save_params(exDir)
            # dis.save_params(exDir)

            #Save images of original and rec
            # save_image(xTest.data, join(exDir, 'original.png'))
            # save_image(recTest.data, join(exDir, 'rec.png'))

            #Save samples
            # sampleDir = join(exDir,'epoch_'+str(e))
            # os.mkdir(sampleDir)
            # print('sample dir:', sampleDir)
            # dae.sample_x(args.M, sampleDir)

In [13]:
# if __name__ == "__main__":
args = Args()

random.seed(args.random_seed)
np.random.seed(args.random_seed)
torch.manual_seed(args.random_seed)

# X, labels = prepare_data(args)
# X = X_train
# labels = y_train
print(X_train.shape)
print(y_train.shape)
trainLoader, testLoader = prepare_dataloaders(args, X_train, X_test, y_train, y_test)

exDir = make_new_folder(args.outDir)
print('Outputs will be saved to:', exDir)


#train_activity_recognition(args, exDir, trainLoader, testLoader)

(46495, 1, 113, 24)
(46495, 5)
Outputs will be saved to: data/experiments/DAE_EVAL_Missing/Ex_9


In [ ]:
train_dae(args, exDir, trainLoader, testLoader)


Experimental details:
root = ../data
batchSize = 64
maxEpochs = 100
nz = 200
lr = 0.0001
fSize = 64
outDir = data/experiments/DAE_EVAL_Missing
commit = eval
alpha = 1.0
M = 5
loss = MSE
loadDAE = False
loadSVM = False
load_DAE_from = None
evalMode = False
comment = 
momentum = 0.1
c = 0.01
svmLR = 0.0001
Ntest = 100
gpuNo = 2
cuda_id = 2
multimodalZ = False
window_len = 512
stride_len = 20
act_list = [1, 2, 3, 4, 5, 6, 7, 12, 13, 16, 17, 24]
imSize = 64
sigma = [8, 10]
random_seed = 2
train_split = 0.8


/home/xyang18/miniconda3/envs/pytorch/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


[0, 0] rec: 0.01394, time: 1.015
[0, 100] rec: 0.00827, time: 0.305
[0, 200] rec: 0.00788, time: 0.322
[0, 300] rec: 0.00818, time: 0.301
[0, 400] rec: 0.00824, time: 0.311
[0, 500] rec: 0.00812, time: 0.435
[0, 600] rec: 0.00780, time: 0.331
[0, 700] rec: 0.00573, time: 0.470
saving params...
[1, 0] rec: 0.00469, time: 0.422
[1, 100] rec: 0.00486, time: 0.316
[1, 200] rec: 0.00397, time: 0.349
[1, 300] rec: 0.00395, time: 0.308
[1, 400] rec: 0.00387, time: 0.338
[1, 500] rec: 0.00366, time: 0.304
[1, 600] rec: 0.00369, time: 0.415
[1, 700] rec: 0.00312, time: 0.343
saving params...
[2, 0] rec: 0.00304, time: 0.374
[2, 100] rec: 0.00316, time: 0.336
[2, 200] rec: 0.00309, time: 0.310
[2, 300] rec: 0.00286, time: 0.319
[2, 400] rec: 0.00295, time: 0.301
[2, 500] rec: 0.00289, time: 0.348
[2, 600] rec: 0.00235, time: 0.372
[2, 700] rec: 0.00227, time: 0.336
saving params...
[3, 0] rec: 0.00281, time: 0.413
[3, 100] rec: 0.00266, time: 0.301
[3, 200] rec: 0.00274, time: 0.431
[3, 300] rec